# View fitted spectra

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-11-27
- last update : 2023-11-27

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import pickle 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  16

## imports

### fitter jaxopt

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

In [ ]:
from dsps.cosmology import DEFAULT_COSMOLOGY, age_at_z

### parameters

In [ ]:
from fors2tostellarpopsynthesis.parameters import SSPParametersFit,paramslist_to_dict

## Configuration

### fitted params

In [ ]:
input_path = "fitssp_results"
input_file = "fitssp_results.h5"

In [ ]:
fullname_input = os.path.join(input_path ,input_file) 

## Read Dataframe 

In [ ]:
df = pd.read_hdf(fullname_input)

In [ ]:
df

In [ ]:
N = len(df)

In [ ]:
df.hist("zobs",bins=30)

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
cNorm = colors.Normalize(0., vmax=1.)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
all_colors = scalarMap.to_rgba(df["zobs"].values, alpha=1)

In [ ]:
bwr_map

In [ ]:
reversed_map

In [ ]:
color_z0 = scalarMap.to_rgba(0., alpha=1)

In [ ]:
color_z1 = scalarMap.to_rgba(1., alpha=1)

In [ ]:
plt.figure(figsize=(10,2))
plt.axvline(0.,color=color_z0,label="z=0")
plt.axvline(1.,color=color_z1,label="z=1")
plt.ylim(0.,1.)
plt.xlim(-0.5,1.5)
plt.legend()

## Decode fitted parameters and plot spectra

In [ ]:
names_fitparams = list(df.columns[8:])

for idx in range(N):


In [ ]:
all_x = []
all_y_nodust = []
all_y_dust = []
the_ymax= 1e-12

for idx in range(N):
    row = df.iloc[idx]
    z_obs = row["zobs"]
    list_params = row[names_fitparams].values
    dict_params = paramslist_to_dict(list_params,names_fitparams)
    x,y_nodust,y_dust = ssp_spectrum_fromparam(dict_params,z_obs)

    ymax = y_nodust.max()
    the_ymax = max(the_ymax,ymax)
    
    all_x.append(x)
    all_y_nodust.append(y_nodust) 
    all_y_dust.append(y_dust) 

ylim_max = the_ymax*2.
ylim_min = ylim_max/1e10

In [ ]:
_, ax = plt.subplots(1, 1)
__= ax.set_yscale('log') 
__= ax.set_xscale('log') 

for idx in range(N):
    ax.plot(all_x[idx],all_y_nodust[idx],color=all_colors[idx])
__= ax.set_xlim(100.,1e5)
__= ax.set_ylim(ylim_min ,ylim_max )

ax.set_xlabel("$\lambda (\\AA)$")
ax.set_ylabel("$L_\\nu(\lambda)$ (AB units - maggies")
ax.set_title("$f_\\nu(\lambda)$ fitted spectra")
ax.grid()


In [ ]:
_, ax = plt.subplots(1, 1)
__= ax.set_yscale('log') 
__= ax.set_xscale('log') 

for idx in range(N):
    ax.plot(all_x[idx],all_y_nodust[idx]/all_x[idx]**2,color=all_colors[idx])
__= ax.set_xlim(100.,1e5)
__= ax.set_ylim(1e-19 ,1e-10 )

ax.set_xlabel("$\lambda (\\AA)$")
ax.set_ylabel("$L_\\lambda(\lambda)$ (AB units - maggies")
ax.set_title("$f_\\lambda(\lambda)$ fitted spectra")
ax.grid()